In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("christophsonntag/OLID")
train_dataset = dataset["train"]
#val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [3]:
for item in test_dataset:
    if item['subtask_a'] == 'NOT':
        print(item['cleaned_tweet'])
        print()

constitutionday revered conservatives hated progressives socialist democrats want change

foxnews nra maga potus trump 2ndamendment rnc usmc veterans fakereporting theswamp dnc liberals fakeoutrage first reduces ca

watching boomer getting news still parole always makes smile wentworth finale treasure

rap form art used express freely gv green light excuse behavior acting like animal streets bx violence way living elevate boo get level longevity queen

5 tips enhance audience connection facebook socialmedia smm

biggbosstamil janani task going first final list

conservatives party low taxation tories tory

nord cannot make single move fully aware happening chill comes spine ghostly rather calm female voice called corner room even time attempt response

constoo place fed conservatives

greatawakening qanon patriotsunited wwg 1wga areyouawake awake see clearly ❤ ️ check video

dayspromo week september 17th days dool coma things gal sami going ok hattie thanks get finishes hattie pretends

### Use BERT to have single vector for sentence

In [4]:
from sentence_transformers import SentenceTransformer
# from transformers import BertTokenizer, BertModel
# import torch
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/dhruv/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
# Load pre-trained BERT model and tokenizer
# model_name = "bert-base-uncased"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

In [6]:
# def get_doc_embedding(document, tokenizer, model):
#     # Tokenize input text
#     inputs = tokenizer(document, return_tensors="pt", padding=True, truncation=True)
#     # Forward pass through the model
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Extract document embedding from hf model output
#     document_embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0)
#     return document_embedding.numpy()

def get_doc_embedding(document, model):
    return model.encode(document)

In [7]:
# Example document from GLUE SST dataset
document = "those so-so films that could have been much better"
doc_emb = get_doc_embedding(document, model)
print(doc_emb.shape)

dummy_vector = 0 * doc_emb


(768,)


### Save the vectors in pkl files

In [10]:
import tqdm
import numpy as np
import pickle

In [11]:
doc_vectors = [] 
# Init dummylist to decode whihc split train/val/test
dummy_list = train_dataset

In [12]:
for item in tqdm.tqdm(dummy_list):
    try:
        sentence = item['cleaned_tweet']
        doc_emb = get_doc_embedding(sentence, model)
    except:
        doc_emb = dummy_vector
    doc_vectors.append(doc_emb)

100%|██████████| 13240/13240 [06:06<00:00, 36.08it/s]


In [13]:
# Open the file and store the reduced sentence vectors
with open('OLID-train-768.pkl', 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump(doc_vectors, file)

### Use PCA to reduce vector dimensions

In [37]:
# doc_vectors_np = np.array(doc_vectors)

# # 768 is emb dimension
# doc_vec_rehsaped = doc_vectors_np.reshape((len(dummy_list),768))

# # Apply PCA for dimensionality reduction
# pca = PCA(n_components = target_dimension)
# pca.fit(doc_vec_rehsaped)

# final_vecs = pca.transform(doc_vec_rehsaped)